# Project 0 - GitHub and Guess Number Game

## Author details

In [1]:
__author__ = "Dmitry Vlasov"
__maintainer__ = "Dmitry Vlasov"
__credits__ = [__author__]
__email__ = "dmitry.v.vlasov@gmail.com"
__year__ = "2020"
__copyright__ = f"\u00a9 Copyright {__year__}, {__author__}"
__license__ = "MIT"
__version__ = '0.1'

## Logging Setup 

In [2]:
import logging
import sys


def setup_logging(log_level):
    """Logging setup

    Parameters:
      log_level (int): minimal level of logged messages
    """
    logformat = "[%(asctime)s] %(levelname)s:%(name)s:%(message)s"
    logging.basicConfig(level=log_level, stream=sys.stdout,
                        format=logformat, datefmt="%Y-%m-%d %H:%M:%S")

    if log_level != None:
        _logger = logging.getLogger(__name__)
        _logger.setLevel(log_level)

In [3]:
_logger = logging.getLogger(__name__)

## Guess Number Game Algorithm Implementations (Guessing Strategies)

Three number guessing strategies are implemented in here:
* `guess_number_game_core_random_snail`, symbolic name - `"random_snail"`;
  **This is the original algoritm implementation which is initially given in the problem statement (`game_core_v2`)**;
* `guess_number_game_core_binary_search`, symbolic name - `"binary-search"`;
  This is a number guessing algoritm implementation based on the binary search algorithm;
* `guess_number_game_core_ternary_search`, symbolic name - `"ternary-search"`;
  This is a number guessing algoritm implementation based on the ternary search algorithm.

The function `score_game` does a test of a given number guessing strategy.
The tested strategy efficiency results are printed in the end of the body of the function `score_game`.
The available strategies are enlisted in the enumeration class `GameCoreType`.

In [4]:
import numpy as np

from enum import Enum
from typing import Tuple, Dict


class GameCoreType(str, Enum):
    OLD_RANDOM_SNAIL = "random-snail",
    BINARY_SEARCH = "binary-search",
    TERNARY_SEARCH = "ternary-search"

    @staticmethod
    def of(value: str):
        for game_core_type in GameCoreType:
            if value == game_core_type:
                return game_core_type
        else:
            return None


def guess_number_game_core_random_snail(number: int, segment: Tuple):
    """This is the original implementation of the INEFFICIENT version of
    the number guessing algorithm (game_core_v2). It is added here as is.

    First, we set any random number, and then we decrease or
    we increase it depending on whether it is more or less than necessary.
    The function takes a guess and returns the number of attempts.
    Like a slow snail randomly thrown on a segment.

    Parameters:
        number (int): a number which the implemented algorithm must guess correctly
        segment (Tuple): the segment the given number belongs to (number ∈ segment)
    """
    count = 0
    iterations = 0
    predicted_number = np.random.randint(segment[0], segment[1])
    while number != predicted_number:  # В этой строке происходит первая попытка сравнения! (count = 0)
        count += 1
        iterations += 1
        if number > predicted_number:
            predicted_number += 1
        elif number < predicted_number:
            predicted_number -= 1
    else:
        count += 1  # Увеличиваем счётчик на 1 для последнего успешного сравнения

    return count, iterations


def guess_number_game_core_binary_search(number: int, segment: Tuple):
    """Binary search based algorithm to guess a number efficiently.

    Parameters:
        number (int): a number which the implemented algorithm must guess correctly
        segment (Tuple): the segment the given number belongs to (number ∈ segment)
    """
    count = 0
    iterations = 0
    lower = segment[0]
    upper = segment[1] + 1

    predicted_number = lower + (upper - lower) // 2
    while number != predicted_number:  # В этой строке происходит первая попытка сравнения! (count = 0)
        count += 1
        iterations += 1
        if number > predicted_number:
            lower = predicted_number + 1
        elif number < predicted_number:
            upper = predicted_number - 1
        predicted_number = lower + (upper - lower) // 2
    else:
        count += 1  # Увеличиваем счётчик на 1 для последнего успешного сравнения

    return count, iterations


def guess_number_game_core_ternary_search(number: int, segment: Tuple):
    """Binary search based algorithm to guess a number efficiently.

    Parameters:
        number (int): a number which the implemented algorithm must guess correctly
        segment (Tuple): the segment the given number belongs to (number ∈ segment)
    """
    count = 0
    iterations = 0
    lower = segment[0]
    upper = segment[1] + 1

    middle_1 = lower + (upper - lower) // 3
    middle_2 = upper - (upper - lower) // 3
    while middle_1 != number and middle_2 != number:  # В этой строке происходит первая попытка сравнения! (count = 0)
        count += 2  # В условии цикла мы сделали сравнение два раза, формально это увеличивает число count на 2.
        iterations += 1  # Также считаем формально число итераций цикла while.
        if number < middle_1:
            upper = middle_1 - 1
        elif number > middle_2:
            lower = middle_2 + 1
        else:
            lower = middle_1 + 1
            upper = middle_2 - 1

        middle_1 = lower + (upper - lower) // 3
        middle_2 = upper - (upper - lower) // 3
    else:
        count += 1 if middle_1 == number else 2  # Увеличиваем значение счётчика для последнего успешного сравнения

    return count, iterations


GAME_CORES = {
    GameCoreType.OLD_RANDOM_SNAIL: guess_number_game_core_random_snail,
    GameCoreType.BINARY_SEARCH: guess_number_game_core_binary_search,
    GameCoreType.TERNARY_SEARCH: guess_number_game_core_ternary_search
}


def get_game_core(game_core_type: GameCoreType):
    """This function give a game core function by a GameCoreType value.
    The dictionary GAME_CORES is used to retrieve a concrete game core.

    Parameters:
        game_core_type (GameCoreType): a game core type 
    """
    return GAME_CORES[game_core_type]


def score_game(game_core_type: GameCoreType,
               attempts: int = 1000,
               segment: Tuple = (1, 100),
               ) -> Dict:
    """Run the game a specified number of times to see how quickly the game guesses a number.

    Parameters:
        game_core_type (GameCoreType): a game core type
        attempts (int): the number of attempts a game algorithm has to demonstrate its efficiency
        segment (Tuple): the segment the given number belongs to (number ∈ segment)

    Returns:
        a dictionary with the structure {"mean-count": N1, "mean-iterations": N2} where {N1, N2} ∈ ℕ.
    """
    _logger.info(f"The guess number game score test: game type - {game_core_type}...")

    efficiency_measuments = list()
    np.random.seed(1)  # We make the RANDOM SEED fixed, in order to make our experiment reproducible!
    random_array = np.random.randint(segment[0], segment[1] + 1, size=attempts)
    game_core = get_game_core(game_core_type)

    for number in random_array:  # The main loop to test the given game core
        efficiency_parameters = game_core(number, segment)
        efficiency_measuments.append(efficiency_parameters)

    # All counts for all guessed numbers
    counts = [efficiency_parameters[0] for efficiency_parameters in efficiency_measuments]
    _logger.debug(f"""Counts for {game_core_type}:\n{counts}""")

    # All loop iterations for all guessed numbers
    iterations = [efficiency_parameters[1] for efficiency_parameters in efficiency_measuments]
    _logger.debug(f"""Iterations for {game_core_type}:\n{iterations}""")

    mean_count = round(np.mean(counts), 1)
    mean_iterations = round(np.mean(iterations), 1)

    
    print(f"- Ваш алгоритм \"{game_core_type.value}\" угадывает число в среднем за\n"
          + f"\t{mean_count} попыток ({int(mean_count)} целых) "
          + f"с {mean_iterations} ({int(mean_iterations)} целых) итерациями основного цикла в среднем.")
    
    _logger.info(f"The guess number game score test: game type - {game_core_type}... Done.")

    return {
        "mean-count": mean_count,
        "mean-iterations": mean_iterations
    }

## Function `main`

The `main` function accepts a set of `kwargs` which are represented in the table below

| kwarg           | Default Value | Description  |
| ---------------:|:--------------|:-------------|
| attempts        | 1000          | Number of testing attempts for a given number guessing algorithm (strategy) |
| segment         | (1, 100)      | A segment of integer numbers where random hidden numbers are searched       |
| game_strategies | `[GameCoreType.OLD_RANDOM_SNAIL, GameCoreType.BINARY_SEARCH, GameCoreType.TERNARY_SEARCH]` | This is a list of game core types (strategies) which can contain values from the enumeration class `GameCoreType` |
| logging_level   | `None`        | A logging level of the log messages, e.g. `logging.INFO`, `logging.DEBUG` |


In [5]:
def main(**kwargs):
    """Main entry point allowing external calls

    Parameters:
      **kwargs : main function keyword arguments
      
    The expected **kwargs:
    - attempts, default value - 1000: Number of testing attempts for a given number guessing algorithm (strategy);
    - segment, default value - (1, 100): A segment of integer numbers where random hidden numbers are searched;
    - game_strategies, default value - [GameCoreType.OLD_RANDOM_SNAIL, GameCoreType.BINARY_SEARCH, GameCoreType.TERNARY_SEARCH]:
      This is a list of game core types (strategies) which can contain values from the enumeration class GameCoreType;
    - logging_level, default value - None: A logging level of the log messages, e.g. logging.INFO, logging.DEBUG.
    """
    kwargs.setdefault("attempts", 1000)
    kwargs.setdefault("segment", (1, 100))
    kwargs.setdefault("game_strategies",
        [GameCoreType.OLD_RANDOM_SNAIL, GameCoreType.BINARY_SEARCH, GameCoreType.TERNARY_SEARCH])
    kwargs.setdefault("logging_level", None)

    setup_logging(kwargs["logging_level"])

    _logger.info("BEGIN - Guess Number Game Strategies Demonstration")

    strategy_efficiencies = dict()
    _logger.info(f"""Selected game strategies to perform testing: {kwargs["game_strategies"]}""")
    for game_core_type in kwargs["game_strategies"]:  # Loop by game_strategies (each one is of type GameCoreType)
        _logger.info(f"\n-----------\nTesting the strategy {game_core_type}...")

        # Calling the score_game function in order to test the strategy named game_core_type.
        efficiency = score_game(game_core_type, kwargs["attempts"], kwargs["segment"])
        strategy_efficiencies[game_core_type] = efficiency

        _logger.info(f"""The strategy efficienty: mean count - {efficiency["mean-count"]},
mean iterations - {efficiency["mean-iterations"]}...""")
        _logger.info(f"\nTesting the strategy {game_core_type}... Done.\n-----------\n")

    # Each item in the dictionary strategy_efficiencies has a form
    # (GameCoreType, {"mean-count": N1, "mean-iterations": N2}) where {N1, N2} ∈ ℕ
    minimal_count_strategy: GameCoreType = min(strategy_efficiencies.items(),
                                               key=lambda it: it[1]["mean-count"])[0]
    minimal_iterations_strategy: GameCoreType = min(strategy_efficiencies.items(),
                                                    key=lambda it: it[1]["mean-iterations"])[0]

    print(f"""Вывод:
\t- наиболее эффективная стратегия по количеству _единичных_ угадываний: {minimal_count_strategy.value};
\t- стратегия с минимальным количеством _итераций_ основного цикла: {minimal_iterations_strategy.value}.""")

    _logger.info("END - Guess Number Game Strategies Demonstration")

## Function `run`

The function `run` prints the program banner, a couple of informational log messages and calls the function main with some concrete `**kwargs`.

In [6]:
BANNER = f"""  __ _ _   _  ___  ___ ___   _ __  _   _ _ __ ___ | |__   ___ _ __ 
 / _` | | | |/ _ \/ __/ __| | '_ \| | | | '_ ` _ \| '_ \ / _ \ '__|
| (_| | |_| |  __/\__ \__ \ | | | | |_| | | | | | | |_) |  __/ |   
 \__, |\__,_|\___||___/___/ |_| |_|\__,_|_| |_| |_|_.__/ \___|_|   
 |___/
 {__copyright__}
 Author email: {__email__}
 License: {__license__}
 Version: {__version__}
 """


def run(**kwargs):
    print(BANNER)
    _logger.info("Running Guess Number Game Strategies Demonstration...")
    main(**kwargs)
    _logger.info("Running Guess Number Game Strategies Demonstration... Done")

---
---

## 1. \[THE MAIN RESULT\] Launching the Guess Number Game Tests with<br/>kwarg default values specified explicitly (excluding the logging level)

Test configuration:
* Attempts: 1000
* Segment: [1, 100]
* Game strategies: {random-snail, binary-search, ternary-search}
* Loggin level: None

In [7]:
run(attempts=1000,
    segment=(1, 100),
    game_strategies=[GameCoreType.OLD_RANDOM_SNAIL, GameCoreType.BINARY_SEARCH, GameCoreType.TERNARY_SEARCH])

  __ _ _   _  ___  ___ ___   _ __  _   _ _ __ ___ | |__   ___ _ __ 
 / _` | | | |/ _ \/ __/ __| | '_ \| | | | '_ ` _ \| '_ \ / _ \ '__|
| (_| | |_| |  __/\__ \__ \ | | | | |_| | | | | | | |_) |  __/ |   
 \__, |\__,_|\___||___/___/ |_| |_|\__,_|_| |_| |_|_.__/ \___|_|   
 |___/
 © Copyright 2020, Dmitry Vlasov
 Author email: dmitry.v.vlasov@gmail.com
 License: MIT
 Version: 0.1
 
- Ваш алгоритм "random-snail" угадывает число в среднем за
	31.9 попыток (31 целых) с 30.9 (30 целых) итерациями основного цикла в среднем.
- Ваш алгоритм "binary-search" угадывает число в среднем за
	5.8 попыток (5 целых) с 4.8 (4 целых) итерациями основного цикла в среднем.
- Ваш алгоритм "ternary-search" угадывает число в среднем за
	7.1 попыток (7 целых) с 2.8 (2 целых) итерациями основного цикла в среднем.
Вывод:
	- наиболее эффективная стратегия по количеству _единичных_ угадываний: binary-search;
	- стратегия с минимальным количеством _итераций_ основного цикла: ternary-search.


---
## 2. Launching the Guess Number Game Tests with the logging level `logging.INFO`

Test configuration:
* Logging level: INFO
* Other settings: default values

In [8]:
run(logging_level=logging.INFO)

  __ _ _   _  ___  ___ ___   _ __  _   _ _ __ ___ | |__   ___ _ __ 
 / _` | | | |/ _ \/ __/ __| | '_ \| | | | '_ ` _ \| '_ \ / _ \ '__|
| (_| | |_| |  __/\__ \__ \ | | | | |_| | | | | | | |_) |  __/ |   
 \__, |\__,_|\___||___/___/ |_| |_|\__,_|_| |_| |_|_.__/ \___|_|   
 |___/
 © Copyright 2020, Dmitry Vlasov
 Author email: dmitry.v.vlasov@gmail.com
 License: MIT
 Version: 0.1
 
[2020-08-10 20:40:22] INFO:__main__:BEGIN - Guess Number Game Strategies Demonstration
[2020-08-10 20:40:22] INFO:__main__:Selected game strategies to perform testing: [<GameCoreType.OLD_RANDOM_SNAIL: 'random-snail'>, <GameCoreType.BINARY_SEARCH: 'binary-search'>, <GameCoreType.TERNARY_SEARCH: 'ternary-search'>]
[2020-08-10 20:40:22] INFO:__main__:
-----------
Testing the strategy random-snail...
[2020-08-10 20:40:22] INFO:__main__:The guess number game score test: game type - random-snail...
- Ваш алгоритм "random-snail" угадывает число в среднем за
	31.9 попыток (31 целых) с 30.9 (30 целых) итерациями основн

---
## 3. Launching the Guess Number Game Tests with the logging level `logging.DEBUG`

Test configuration:
* Logging level: DEBUG
* Other settings: default values

In [9]:
run(logging_level=logging.DEBUG)

  __ _ _   _  ___  ___ ___   _ __  _   _ _ __ ___ | |__   ___ _ __ 
 / _` | | | |/ _ \/ __/ __| | '_ \| | | | '_ ` _ \| '_ \ / _ \ '__|
| (_| | |_| |  __/\__ \__ \ | | | | |_| | | | | | | |_) |  __/ |   
 \__, |\__,_|\___||___/___/ |_| |_|\__,_|_| |_| |_|_.__/ \___|_|   
 |___/
 © Copyright 2020, Dmitry Vlasov
 Author email: dmitry.v.vlasov@gmail.com
 License: MIT
 Version: 0.1
 
[2020-08-10 20:40:22] INFO:__main__:Running Guess Number Game Strategies Demonstration...
[2020-08-10 20:40:22] INFO:__main__:BEGIN - Guess Number Game Strategies Demonstration
[2020-08-10 20:40:22] INFO:__main__:Selected game strategies to perform testing: [<GameCoreType.OLD_RANDOM_SNAIL: 'random-snail'>, <GameCoreType.BINARY_SEARCH: 'binary-search'>, <GameCoreType.TERNARY_SEARCH: 'ternary-search'>]
[2020-08-10 20:40:22] INFO:__main__:
-----------
Testing the strategy random-snail...
[2020-08-10 20:40:22] INFO:__main__:The guess number game score test: game type - random-snail...
[2020-08-10 20:40:22] DEBUG:_

- Ваш алгоритм "random-snail" угадывает число в среднем за
	31.9 попыток (31 целых) с 30.9 (30 целых) итерациями основного цикла в среднем.
[2020-08-10 20:40:22] INFO:__main__:The guess number game score test: game type - random-snail... Done.
[2020-08-10 20:40:22] INFO:__main__:The strategy efficienty: mean count - 31.9,
mean iterations - 30.9...
[2020-08-10 20:40:22] INFO:__main__:
Testing the strategy random-snail... Done.
-----------

[2020-08-10 20:40:22] INFO:__main__:
-----------
Testing the strategy binary-search...
[2020-08-10 20:40:22] INFO:__main__:The guess number game score test: game type - binary-search...
[2020-08-10 20:40:22] DEBUG:__main__:Counts for binary-search:
[3, 6, 7, 6, 2, 4, 6, 7, 7, 7, 6, 5, 6, 6, 1, 5, 6, 5, 3, 6, 7, 5, 1, 4, 7, 7, 4, 7, 6, 7, 6, 7, 6, 7, 6, 6, 7, 6, 6, 4, 5, 3, 7, 7, 7, 4, 5, 6, 7, 7, 7, 6, 6, 4, 2, 4, 6, 7, 7, 7, 6, 6, 6, 7, 4, 6, 7, 7, 7, 7, 6, 6, 6, 7, 7, 5, 3, 6, 7, 6, 5, 7, 7, 7, 7, 6, 7, 6, 5, 6, 7, 3, 5, 7, 3, 7, 7, 5, 7, 6, 6, 7, 6

[2020-08-10 20:40:22] DEBUG:__main__:Iterations for ternary-search:
[2, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 2, 2, 3, 4, 4, 2, 4, 3, 3, 2, 2, 4, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 3, 1, 3, 4, 3, 2, 3, 4, 4, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 2, 3, 4, 3, 4, 2, 2, 4, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 0, 3, 3, 2, 3, 3, 3, 4, 3, 3, 1, 3, 3, 3, 3, 2, 4, 2, 3, 4, 3, 4, 3, 3, 4, 3, 2, 3, 3, 2, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 4, 2, 4, 4, 3, 4, 4, 2, 2, 4, 2, 2, 3, 4, 3, 1, 4, 3, 2, 2, 3, 3, 2, 2, 4, 3, 3, 3, 4, 4, 2, 1, 3, 2, 3, 4, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 4, 3, 2, 3, 1, 3, 3, 4, 3, 3, 2, 3, 4, 3, 3, 2, 3, 3, 3, 3, 3, 2, 1, 3, 3, 4, 3, 3, 0, 3, 1, 4, 3, 2, 2, 3, 4, 3, 3, 0, 3, 4, 3, 2, 4, 3, 3, 3, 2, 4, 3, 2, 2, 4, 4, 3, 4, 4, 3, 4, 3, 3, 3, 3, 4, 4, 1, 3, 3, 3, 2, 3, 2, 2, 3, 4, 3, 3, 2, 3, 3, 3, 0, 4, 3, 2, 3, 3, 3, 3, 2, 3, 3, 4, 2, 3, 4, 3, 3, 3, 3, 3, 2, 1, 3, 3, 4, 3, 3, 1, 3, 3, 2, 3, 3, 3, 2, 4, 3, 1, 4, 3, 2, 4, 3, 3, 4, 3, 1, 3, 3, 2

---
---
*Thank you for your patience*
![Thank you for your patience](https://i.pinimg.com/originals/51/06/97/51069722611596d26e97e6079a71d6d9.jpg)